Insert explanation here. Allow 15 minutes for completion.

In [1]:
load("preamble.sage")

## The orthogonal group

Produce a copy of $\mathrm{SO}(10)$ with invariant form
$$
x_1 x_6 + x_2 x_7 + x_3 x_8 + x_4 x_9 + x_5 x_{10}.
$$

In [2]:
F = GF(2)

In [3]:
G = SO(10,2,e=1)
G.order()

46998591897600

In [4]:
J = G.invariant_form(); J

[0 1 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 1 0]

In [5]:
J1 = block_matrix(2,2, [0, identity_matrix(F, 5), identity_matrix(F, 5), 0], subdivide=False)
Pg = SymmetricGroup(10)([(2,6,8,9,5,3),(4,7)])
l1 = []
for g in G.gens():
    M = copy(g.matrix())
    M.permute_rows_and_columns(~Pg, ~Pg)
    assert M*J1*M.transpose() == J1
    l1.append(M)
G0 = GL(10, F).subgroup(l1)

In [6]:
assert all(g*J1*g.matrix().transpose() == J1 for g in G0.gens())

Construct the index-2 subgroup of $\mathrm{SO}$ stabilizing one component of $OG$.

In [7]:
l2 = [identity_matrix(10, F)]
l2[0][3,9] = 1
l2[0][4,8] = 1
l2.append(Matrix(F, [[0,1,0,0,0,1,0,0,1,0],
                     [0,0,0,1,1,0,1,1,0,0],
                     [0,1,0,0,0,1,0,1,0,1],
                     [1,0,1,0,1,0,1,1,0,1],
                     [0,1,0,1,1,0,1,0,1,0],
                     [0,0,1,0,0,0,1,0,0,1],
                     [1,1,0,1,1,0,0,0,0,0],
                     [0,0,0,0,1,0,0,1,0,0],
                     [0,1,0,0,0,1,0,1,1,0],
                     [1,0,0,1,1,0,0,1,0,0]]))
G1 = G0.subgroup(l2)
assert G0.order() // G1.order() == 2

## Lagrangian subspaces

Apply a group element to a Lagrangian subspace represented by an echelon matrix.

## The orthogonal Grassmannian

Construct the Plucker relations for $OG$ in terms of Pfaffians.

In [8]:
P.<x0,x12,x13,x14,x15,x23,x24,x25,x34,x35,x45,x1234,x1235,x1245,x1345,x2345> = PolynomialRing(F, 16)
quads = [x0*x2345 + x23*x45 + x24*x35 + x25*x34,
         x12*x1345 + x13*x1245 + x14*x1235 + x15*x1234,
         x0*x1345 + x13*x45 + x14*x35 + x15*x34,
         x12*x2345 + x23*x1245 + x24*x1235 + x25*x1234,
         x0*x1245 + x12*x45 + x14*x25 + x15*x24,
         x13*x2345 + x23*x1345 + x34*x1235 + x35*x1234,
         x0*x1235 + x12*x35 + x13*x25 + x15*x23,
         x14*x2345 + x24*x1345 + x34*x1245 + x45*x1234,
         x0*x1234 + x12*x34 + x13*x24 + x14*x23,
         x15*x2345 + x25*x1345 + x35*x1245 + x45*x1235]

Lagrangian subspaces in $\mathbb{F}_2^{10}$ are represented by $5 \times 10$ echelon matrices.

Given a Lagrangian subspace of $\mathbb{F}_2^{10}$, compute the *spinor coordinate* in $S^+$. If `verify` is `True`, we check that the intersection is one-dimensional and that the resulting coordinate satisfies the Plucker relations.

In [9]:
def spinor_coordinate(M, verify=True, set_immutable=True):
    coord1 = [_ for i in [0,2,4] for _ in itertools.combinations(range(5), i)]
    coord2 = [_ for i in [1,3,5] for _ in itertools.combinations(range(5), i)]
    ops = []
    # Annihilation operators.
    for i in range(5):
        ops.append(Matrix(F, 16, 16, [[1 if 
                                       i not in coord2[k] and set(coord2[k]).union(set([i])) == set(coord1[j]) 
                                       else 0 for k in range(16)] for j in range(16)]))
    # Creation operators.
    for i in range(5):
        ops.append(Matrix(F, 16, 16, [[1 if 
                                       i not in coord1[j] and set(coord1[j]).union(set([i])) == set(coord2[k]) 
                                       else 0 for k in range(16)] for j in range(16)]))
    l = [sum(ops[j]*M[i,j] for j in range(10)) for i in range(5)]
    l2 = [N.right_kernel() for N in l]
    K = l2[0]
    for V in l2:
        K = K.intersection(V)
    if K.dimension() != 1:
        if verify:
            raise ValueError
        else:
            return None
    v = K.gens()[0]
    if verify:
        assert all(P(*v) == 0 for P in quads)
    if set_immutable:
        v.set_immutable()
    return v

Given spinor coordinates of some Lagrangian subspaces, compute the intersection of their linear span with $OG$ as an ideal in $P$.

In [10]:
def linear_section(coords):
    V = Matrix(coords).right_kernel()
    tmp2 = [sum(P.gens()[i] * v[-1-i] for i in range(16)) for v in V.gens()] + quads
    return P.ideal(tmp2)

## Points of the orthogonal Grassmannian

Construct a point on $OG$.

In [11]:
M0 = block_matrix(1, 2, [identity_matrix(F, 5), Matrix(F,5)], subdivide=False)
M0.set_immutable()
print(M0)
print(spinor_coordinate(M0))
assert M0.echelon_form() == M0

[1 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0]
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)


Construct the orbit of $M_0$ under the action of (conjugated) $\mathrm{SO}(10)$.

In [12]:
def apply_group_elem(g, M, set_immutable=False):
    M1 = (M*~g).echelon_form()
    if set_immutable:
        M1.set_immutable()
    return M1

In [13]:
vertices = []
new_vertices = [M0]
while new_vertices:
    M = new_vertices.pop()
    vertices.append(M)
    for g in G1.gens():
        M1 = apply_group_elem(g, M, set_immutable=True)
        if M1 not in vertices + new_vertices:
            new_vertices.append(M1)
print(len(vertices))

2295


Compute spinor coordinates of these spaces (with verification).

In [14]:
S = vertices
coords = {M: spinor_coordinate(M) for M in S}

## Computation of orbits

Forbid orbits of Lagrangian subspaces according to the following criteria:  
- No two can have intersection of dimension $\neq 1$.
- No three can have coplanar spinor coordinates.
- No five or six can have spinor coordinates whose linear span has positive-dimensional intersection with $OG$.
- No six can have spinor cordinates whose linear span meets $OG$ in $> 7$ rational points.

In [15]:
def forbid(mats, easy=False, coords=coords):
    if len(mats) == 2:
        vecs = [M.row_space() for M in mats]
        if vecs[0].intersection(vecs[1]).dimension() != 1:
            return True
    tmp = [coords[M] for M in mats]
    if len(mats) == 3:
        if Matrix(tmp).rank() < 3:
            return True
    if len(mats) in [5,6]:
        J = linear_section(tmp)
        if J.dimension() > 1:
            return True
    if easy:
        return False
    if len(mats) == 6:
        tmp2 = [(J2.radical().hilbert_polynomial(), J2.hilbert_polynomial()) \
             for J2 in J.primary_decomposition()]
        if sum(1 for (a,b) in tmp2 if a==1) >= 8:
            return True
    return False

Compute an orbit lookup tree of depth 6.

In [16]:
def optimized_rep(g):
    return g.matrix()

In [17]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': vec_stab,
           'optimized_rep': optimized_rep,
           'forbid': forbid}

In [18]:
tree = build_orbit_tree(G1, vertices, 6, methods, verbose=False)

A
A
A
A
A


Write spinor coordinates of representative 6-tuples to a file.

In [19]:
l = [[coords[M] for M in mats] for mats in green_nodes(tree, 6)]
with open("6-tuples.txt", "w") as f:
    f.write(str(l))

In [20]:
len(l)

494

In [21]:
print("This stage complete!")

This stage complete!
